## 线性回归的简洁实现

### 通过使用深度学习框架来简洁地实现线性回归模型、生成数据集

In [56]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)


## 调用框架中现有的API来读取数据

In [57]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    # 将数据集封装到Dataset中
    dataset = data.TensorDataset(*data_arrays)
    # 将数据集通过DataLoader加载（根据批量、是否打散数据来加载）
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[ 0.1177,  0.1816],
         [ 0.3283, -1.7649],
         [-2.3040, -0.2416],
         [-1.6857, -1.6319],
         [-1.6701, -0.2968],
         [-0.1680, -0.4173],
         [ 0.2294,  2.6757],
         [ 0.2440, -1.1210],
         [-0.7817,  0.6410],
         [ 0.8900,  1.4798]]),
 tensor([[ 3.8158],
         [10.8573],
         [ 0.4306],
         [ 6.3501],
         [ 1.8683],
         [ 5.2815],
         [-4.4254],
         [ 8.5049],
         [ 0.4630],
         [ 0.9565]])]

## 使用框架的预定义好的层

In [45]:
# `nn`是神经网络（Neural Network）的缩写
from torch import nn
## nn.Linear 使用线性层，或者全连接层
## Sequential: list of layers
net = nn.Sequential(nn.Linear(2,1))

## 初始化模型参数

In [46]:
# 线性模型只有一个leayer，所以net[0]就能拿到该层， net[0].weight是参数（w),
# 使用正态分布替换data的值(均值为0，方差为0.01）
net[0].weight.data.normal_(0, 0.01)
# bias是偏差，设置为0
net[0].bias.data.fill_(0)

tensor([0.])

## 计算均方误差使用的是```MSELoss```类，也称平方$L_2$范数

In [47]:
loss = nn.MSELoss()

## 实例化```SGD```实例

In [60]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练过程与我们从零开始时所做的非常相似

In [61]:
# data_iter = load_array((features, labels), batch_size)
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        # 清零梯度
        trainer.zero_grad()
        # 计算backward
        l.backward()
        # 更新模型
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000099
epoch 2, loss 0.000100
epoch 3, loss 0.000100
